### Prelude:

In [1]:
import numpy as np
import pandas as pd

/var/folders/ys/2yh_kw0938l96bcd357qq56w0000gn/T/ipykernel_74099/1662815981.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
data = pd.read_csv("./../Datasets/SOSC 13300 Severson and Coleman Extension Survey_April 22, 2024_09.12.csv")

### Data Cleaning:

In [3]:
#drop rows where survey was not completed
dataClean = data[data['Finished'] == 'True']

#Drop Unnecessary Columns
dataClean = data.drop(columns=['StartDate', 'EndDate', 'Status', 'IPAddress', 'Progress', 'RecordedDate', 'ResponseId', 'RecipientLastName',
                               'RecipientFirstName', 'RecipientEmail', 'ExternalReference', 'LocationLatitude', 'LocationLongitude', 'DistributionChannel', 
                               'InfCons', 'participantId', 'assignmentId', 'projectId'])

#dropping first and 2nd rows
dataClean = dataClean.drop([0, 1]).reset_index()

#drop rows where participant failed attention check
dataClean = dataClean[dataClean['Attention_Check_1'] == 'Strongly like']
dataClean = dataClean[dataClean['Attention_Check_2'] == '1,3']

In [4]:
#making complete party_id number (merging party_id columns)

#making columns into integers
dataClean['QID74'] = dataClean['QID74'].replace(float('nan'), 0).astype(int)
dataClean['party_id.1'] = dataClean['party_id.1'].replace(float('nan'), 0).astype(int)
dataClean['party_id.2'] = dataClean['party_id.2'].replace(float('nan'), 0).astype(int)

dataClean['party_id_merged'] = dataClean['QID74'] + dataClean['party_id.1'] + dataClean['party_id.2']

In [5]:
#making average climate support column (mean of Gas Tax, Carbon Tax, and Renewable Energy)
columns = ['GasTax', 'CarbTax', 'Treaty', 'RegCarb', 'GasTax_after', 'CarbTax_after', 'Treaty_after', 'RegCarb_after']
for i in columns:
    #ASSUMING 0 MEANS PREFER NOT TO ANSWER
    dataClean[i] = dataClean[i].replace(float('nan'), 0).astype(int).replace(0, np.nan)


dataClean['mean_climate_support_before'] = dataClean[['GasTax', 'CarbTax', 'Treaty', 'RegCarb']].mean(axis=1)
dataClean['mean_climate_support_after'] = dataClean[['GasTax_after', 'CarbTax_after', 'Treaty_after', 'RegCarb_after']].mean(axis=1)

In [6]:
#save cleaned data
dataClean.to_csv('./../Datasets/cleaned_data.csv', index=False)

### Questions about Data Collection:
- What is the 0 value for the climate policy choices (no opinion?)
- What are the different treatment effects/what do they correspond to?